In [ ]:
"""
# TODO: 1. Открыть https://www.moex.com (https://www.moex.com/)
# TODO: 2. Нажать на кнопку Меню, выбрать Срочный рынок, далее выбрать Индикативные курсы.
# TODO: 3. В выпадающем списке выбирать валюты: USD/RUB - Доллар США к российскому рублю.
# TODO: 4. Когда откроется динамика курса, скопировать в excel курс за последний текущий месяц.
# TODO: 5. В excel шапка (A) Дата, (B) Курс, (C)Изменение.
# TODO: 6. Повторить для Евро (EUR/RUB - Евро к российскому рублю), записать в ячейки (D), (E), (F) дату евро, курс, изменение.
# TODO: 7. Для каждой строки полученного файла поделить курс евро на доллар и полученное значение записать в ячейку (G).
# TODO: 8. Выровнять – автоширина.
# TODO: 9. Формат чисел – финансовый.
# TODO: 10. Проверить, чтобы автосумма в excel опознавала ячейки как числовой формат.
# TODO: 11. Послать итоговый файл отчета себе на почту.
# TODO: 12. В письме указать количество строк в excel в правильном склонении."""


import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import re
import requests
import xmltodict
import xlsxwriter
from bs4 import BeautifulSoup
import urllib.request
import lxml.html as LH
import html2text
import openpyxl
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter
import smtplib                                             
import os                                                   
import mimetypes                                            
from email import encoders                                  
from email.mime.base import MIMEBase                        
from email.mime.text import MIMEText                        
from email.mime.image import MIMEImage                      
from email.mime.audio import MIMEAudio                      
from email.mime.multipart import MIMEMultipart 
import pandas as pd


# Подключаем драйвер Chrome
driver = webdriver.Chrome(executable_path=r'C:\chromedriver\chromedriver.exe')

# TODO: 1. Открыть https://www.moex.com (https://www.moex.com/)
# Указываем ссылку и переходим на нее
URL = 'https://www.moex.com/'
driver.get(URL)


# TODO: 2. Нажать на кнопку Меню, выбрать Срочный рынок, далее выбрать Индикативные курсы.
driver.implicitly_wait(5)

# Нажать на кнопку Меню
elem_menu = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div/div/div[1]/div/div[2]/a[1]').click()

driver.implicitly_wait(5)

# Срочный рынок
elem_menu_market = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div/div/div[1]/div/div[2]/div[4]/div[2]/div[1]/a').click()
elem_menu_market_value = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div/div/div[1]/div/div[2]/div[4]/div[2]/div[1]/div/div[3]/a').click()

driver.implicitly_wait(5)

# Согласие с условиями
elem_alright = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div/div[1]/div/a[1]').click()

driver.implicitly_wait(5)

# Индикативные курсы
elem_courses = driver.find_elements_by_xpath('//*[@id="ctl00_frmLeftMenuWrap"]/div/div/div/div[1]/div/a')[0].click()
elem_courses = driver.find_element_by_xpath('//*[@id="ctl00_frmLeftMenuWrap"]/div/div/div/div[2]/div/a[13]/div/span/div').click()


# TODO: 3. В выпадающем списке выбирать валюты: USD/RUB - Доллар США к российскому рублю.
path_excel_url = driver.current_url

"""
Возникли проблемы с тем, чтобы заставить работать элемент селект, поэтому решение "обходное"

По идее одна из требуемых отношений валют выбрана по умолчанию, а вторая нам известна заранее.
Тем не менее, нижепредставленный код работает в более общем виде:
мы получаем список всех отношений и возможность к ним обращаться по неоходимости
"""

#Функция выбора валюты
def currency_change(string):
    element = driver.find_element_by_xpath('//*[@id="ctl00_PageContent_CurrencySelect"]')
    all_options = element.find_elements_by_tag_name("option")
    currencies = [option.get_attribute("value") for option in all_options]
    current_currency = path_excel_url[path_excel_url.find('=') + 1:]
    driver.get(path_excel_url.replace(current_currency,currencies[currencies.index(string)]))

#Выбираем нужную валюту(USD/RUB)
currency_change('USD_RUB')

path_excel_url = driver.current_url


#Создание датафрейма с данными по выбранной валюте за последний текущий месяц
def wanted_slice():
    df = pd.read_html(path_excel_url, thousands=None)[2]
    data_month = df['Дата']['Дата'][0][3:5]
    from_data = f'01.{data_month}.2021'
    newest_data = df[:df[df['Дата']['Дата'] == from_data].index[0] + 1]
    memory_data = df[:df[df['Дата']['Дата'] == from_data].index[0] + 2]
    newest_data.loc[:,('Курс основного клиринга','Значение')] = newest_data.loc[:,('Курс основного клиринга','Значение')].apply(lambda x: x.replace(',','.')).astype(float)
    memory_data.loc[:,('Курс основного клиринга','Значение')] =  memory_data.loc[:,('Курс основного клиринга','Значение')].apply(lambda x: x.replace(',','.')).astype(float)
    return (memory_data, newest_data)

#Датафрейм для USD/RUB
memory_data1, newest_data1 = wanted_slice()

#Выбираем нужную валюту(EUR/RUB)
currency_change('EUR_RUB')

path_excel_url = driver.current_url

#Датафрейм для EUR/RUB
memory_data2, newest_data2 = wanted_slice()

#Изменяем названия столбцов, чтобы избежать ошибки multiindex
memory_data2 = memory_data2.rename(columns={'Дата':'Дата1', 'Курс основного клиринга': 'Курс основного клиринга_1', 'Курс промежуточного клиринга': 'Курс промежуточногоо клиринга_1'})
newest_data2 = newest_data2.rename(columns={'Дата':'Дата1','Курс основного клиринга': 'Курс основного клиринга_1', 'Курс промежуточного клиринга': 'Курс промежуточногоо клиринга_1'})

#Соединяем данные обоих валютных отношений в один датафрейм
memory_data = pd.concat([memory_data1, memory_data2], axis = 1)
newest_data = pd.concat([newest_data1, newest_data2], axis = 1)

#Функция создания файла EXCEL с данными сводного датафрейма 
def scrap_to_excel():
    workbook = xlsxwriter. Workbook('E:/Anaconda/Cos/Currency.xlsx')
    worksheet = workbook. add_worksheet('Main')
    
# TODO: 5. В excel шапка (A) Дата, (B) Курс, (C)Изменение.    
    worksheet.write(0, 0, 'Дата')
    worksheet.write(0, 1, 'Курс')
    worksheet.write(0, 2, 'Изменение')
    worksheet.write(0, 3, 'Дата')
    worksheet.write(0, 4, 'Курс')
    worksheet.write(0, 5, 'Изменение')
    
# TODO: 4. Когда откроется динамика курса, скопировать в excel курс за последний текущий месяц.    
    range_for = newest_data['Курс основного клиринга']['Значение'].shape[0]
    for i in range(range_for):
        worksheet.write(i + 1, 0, newest_data['Дата']['Дата'][i])
    for i in range(range_for):
        worksheet.write(i + 1, 1, round(newest_data['Курс основного клиринга']['Значение'][i], 2))
    for i in range(range_for):
        worksheet.write(i + 1, 2, round(memory_data['Курс основного клиринга']['Значение'][i] - memory_data['Курс основного клиринга']['Значение'][i + 1], 2))


# TODO: 6. Повторить для Евро (EUR/RUB - Евро к российскому рублю), записать в ячейки (D), (E), (F) дату евро, курс, изменение.    
    for i in range(range_for):
        worksheet.write(i + 1, 3, newest_data['Дата1']['Дата1'][i])
    for i in range(range_for):
        worksheet.write(i + 1, 4 , round(newest_data['Курс основного клиринга_1']['Значение'][i], 2))
    for i in range(range_for):
        worksheet.write(i + 1, 5, round(memory_data['Курс основного клиринга_1']['Значение'][i] - memory_data['Курс основного клиринга_1']['Значение'][i + 1], 2))
    
# TODO: 7. Для каждой строки полученного файла поделить курс евро на доллар и полученное значение записать в ячейку (G).    
    for i in range(range_for):
        worksheet.write(i + 1, 6,newest_data['Курс основного клиринга']['Значение'][i])
        
    workbook.close()
    
#Создание требуемого EXCEL-файла
scrap_to_excel()

# TODO: 8. Выровнять – автоширина.
wb = openpyxl.load_workbook("E:/Anaconda/Cos/Currency.xlsx")
ws = wb["Main"]

dim_holder = DimensionHolder(worksheet=ws)

for col in range(ws.min_column, ws.max_column + 1):
    dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, bestFit=True)

ws.column_dimensions = dim_holder


# TODO: 9. Формат чисел – финансовый. Задается округлением функции scrap_to_excel и числовым форматом ячеек.
# TODO: 10. Проверить, чтобы автосумма в excel опознавала ячейки как числовой формат. 
#Выполнить не успел, но openpyxl.styles.numbers.BUILTIN_FORMATS[2] задает числовой формат ячеек
for column in ['B', 'C', 'E', 'F', 'G']:
    for row in range(1, ws.max_row+1):
        ws["{}{}".format(column, row)].number_format = openpyxl.styles.numbers.BUILTIN_FORMATS[2]

wb.save('E:/Anaconda/Cos/Currency.xlsx')


# TODO: 12. В письме указать количество строк в excel в правильном склонении.
def mail_text(n): 
    es = ['а', 'и', '']
    n = n % 100
    if n>=11 and n<=19:
        s=es[2] 
    else:
        i = n % 10
        if i == 1:
            s = es[0] 
        elif i in [2,3,4]:
            s = es[1] 
        else:
            s = es[2] 
    return f'{n}' +' строк' + s 

mail_body = mail_text(range_for)


# TODO: 11. Послать итоговый файл отчета себе на почту. В целях безопасности почтовые адреса и пароли исключены из кода
send_from = "sender _mail_address@gmail.com"
send_to = "receiver_mail_address@greenatom.ru"

msg = MIMEMultipart()

msg['From'] = send_from
msg['To'] = send_to
msg['Subject'] = "Test task"

body = mail_body

msg.attach(MIMEText(body, 'plain'))

filename = "Currency.xlsx"
attachment = open("E:/Anaconda/Cos/Currency.xlsx", "rb")

part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

msg.attach(part)

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(send_from, "sender_mail_password")
text = msg.as_string()
server.sendmail(send_from, send_to, text)
server.quit()

#Закрываем браузер после успешного выполнения кода
driver.quit()